In [91]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime as dt
import datetime, os, wget, requests, unidecode
import uuid
import xlwings as xw
from bs4 import BeautifulSoup

In [15]:
start_date = dt(2020, 11, 8)
end_date = dt(2021, 1, 26)
hours = ['14', '17', '21', '04', '08', '11']
last_day_hours = ['04', '08', '11', '14']

In [16]:
def download_excel(date, hour):
    str_date = date.strftime("%Y%m%d")
    url = f'https://www.vendeeglobe.org/download-race-data/vendeeglobe_{str_date}_{hour}0000.xlsx'
    try:
        if not os.path.exists(f'vg_data/vendeeglobe_{str_date}_{hour}0000.xlsx'):
            wget.download(url, f'vg_data/vendeeglobe_{str_date}_{hour}0000.xlsx')
    except Exception as e:
        print(f'Error for {date.strftime("%Y-%m-%d")} at {hour}H:')
        print(e)

In [19]:
date = start_date
while date <= end_date:
    str_date = date.strftime("%Y%m%d")
    for hour in hours:
        download_excel(date, hour)
    date += datetime.timedelta(days=1)
for hour in last_day_hours:
    download_excel(date, hour)

Error for 2020-11-08 at 04H:
HTTP Error 404: Not Found
Error for 2020-11-08 at 08H:
HTTP Error 404: Not Found
Error for 2020-11-08 at 11H:
HTTP Error 404: Not Found


Faut il lire le premier ? Tous les compétiteurs sont au meme point de départ

In [95]:
def clean_excel(date, hour):
    str_date = date.strftime("%Y%m%d")
    data = None
    if os.path.exists(f'vg_data/vendeeglobe_{str_date}_{hour}0000.xlsx'):
        with xw.Book(f"vg_data/vendeeglobe_{str_date}_{hour}0000.xlsx", mode="r") as book:
            sheet1 = book.sheets[0]
            data = sheet1.cells.options("df").value

        data = data.iloc[2:]
        data.iloc[1,:3] = data.iloc[0,:3]
        data.iloc[1,-2:] = data.iloc[0,-2:]
        data.iloc[1] = data.iloc[1].str.split("\r\n").str[0]
        data.iloc[1, 1] = data.iloc[1, 1].replace(" / ",' ')
        data.iloc[1, 2] = data.iloc[1, 2].replace(" / ",'\r\n')
        data.iloc[1,6:10] = data.iloc[1,6:10]+"_30m"
        data.iloc[1,10:14] = data.iloc[1,10:14]+"_last"
        data.iloc[1,14:18] = data.iloc[1,14:18]+"_24h"
        data.columns = data.iloc[1]
        data = data.iloc[2:]
        data = data.iloc[:-4]
        nat_voile = "(?P<Nationalite>.*) (?P<Voile>[0-9]*)"
        data[["Nationalité", "Voile"]] = data['Nat. Voile'].str.extract(nat_voile, expand=True)
        skipper_bateau = "(?P<Skipper>.*)\r\n(?P<nBateau>.*)"
        data[["Skipper", "Bateau"]] = data['Skipper\r\nBateau'].str.extract(skipper_bateau, expand=True)
        data = data.drop(['Nat. Voile', 'Skipper\r\nBateau'], axis=1)
        data["Skipper"] = data["Skipper"].map(lambda x: unidecode.unidecode(x.lower()))
        data["date"] = date 
        data = data.reset_index(drop=True)
    return data

In [115]:
date = start_date
race_df = pd.DataFrame()
while date <= end_date + datetime.timedelta(days=1):
    for hour in hours:
        tmp_df = clean_excel(date, hour)
        try :
            race_df = pd.concat([race_df, tmp_df])
        except Exception as e:
            print(e)
            break
    date += datetime.timedelta(days=1)
race_df = race_df.dropna(subset=race_df.columns[1:18]).reset_index(drop=True)
race_df.head()

,Rang,Heure FR,Latitude,Longitude,Cap_30m,Vitesse_30m,VMG_30m,Distance_30m,Cap_last,Vitesse_last,...,Vitesse_24h,VMG_24h,Distance_24h,DTF,DTL,Nationalité,Voile,Skipper,Bateau,date
0,1,15:30 FR\r\n,46°24.46'N,01°50.48'W,241°,17.7 kts,17.5 kts,0.3 nm,357°,0.0 kts,...,0.3 kts,0.3 kts,6.1 nm,24293.9 nm,0.0 nm,FRA,18,louis burton,Bureau Vallée 2,2020-11-08
1,2,15:31 FR\r\n1min,46°24.34'N,01°49.82'W,241°,11.1 kts,10.9 kts,0.4 nm,357°,0.0 kts,...,0.3 kts,0.2 kts,6.0 nm,24294.2 nm,0.4 nm,MON,10,boris herrmann,Seaexplorer - Yacht Club De Monaco,2020-11-08
2,3,15:30 FR\r\n,46°24.91'N,01°49.99'W,244°,15.5 kts,15.5 kts,0.5 nm,357°,0.0 kts,...,0.2 kts,0.2 kts,5.5 nm,24294.3 nm,0.5 nm,FRA,8,jeremie beyou,Charal,2020-11-08
3,4,15:30 FR\r\n,46°24.71'N,01°49.68'W,244°,13.2 kts,13.1 kts,0.7 nm,357°,0.0 kts,...,0.2 kts,0.2 kts,5.6 nm,24294.5 nm,0.6 nm,FRA,59,thomas ruyant,LinkedOut,2020-11-08
4,5,15:30 FR\r\n,46°24.59'N,01°49.56'W,246°,10.9 kts,10.9 kts,0.2 nm,357°,0.0 kts,...,0.8 kts,0.7 kts,5.8 nm,24294.5 nm,0.6 nm,FRA,53,maxime sorel,V And B Mayenne,2020-11-08


In [117]:
race_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13671 entries, 0 to 13670
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Rang           13671 non-null  object        
 1   Heure FR       13671 non-null  object        
 2   Latitude       13671 non-null  object        
 3   Longitude      13671 non-null  object        
 4   Cap_30m        13671 non-null  object        
 5   Vitesse_30m    13671 non-null  object        
 6   VMG_30m        13671 non-null  object        
 7   Distance_30m   13671 non-null  object        
 8   Cap_last       13671 non-null  object        
 9   Vitesse_last   13671 non-null  object        
 10  VMG_last       13671 non-null  object        
 11  Distance_last  13671 non-null  object        
 12  Cap_24h        13671 non-null  object        
 13  Vitesse_24h    13671 non-null  object        
 14  VMG_24h        13671 non-null  object        
 15  Distance_24h   1367

In [40]:
url = "https://www.vendeeglobe.org/fr/glossaire"
page = requests.get(url).content
soup = BeautifulSoup(page)

In [41]:
specs_ul = soup.find_all('ul', {"class","boats-list__popup-specs-list"})

In [114]:
dict_list = []
for spec_ul in specs_ul:
    specs_list = [elem.string.split(' : ') for elem in spec_ul.find_all('li')]
    specs_dict = {elem[0]: elem[1] for elem in specs_list}
    skipper = spec_ul.parent()[0].parent()[-5]['href'].split('/')[-1].replace('-', ' ')
    specs_dict["Skipper"] = skipper
    dict_list.append(specs_dict)
specs_df = pd.DataFrame(dict_list)
specs_df.head()

,Numéro de voile,Anciens noms du bateau,Architecte,Chantier,Date de lancement,Longueur,Largeur,Tirant d'eau,Déplacement (poids),Nombre de dérives,Hauteur mât,Voile quille,Surface de voiles au près,Surface de voiles au portant,Skipper
0,FRA 56,"No Way Back, Vento di Sardegna",VPLP/Verdier,Persico Marine,01 Août 2015,"18,28 m","5,85 m","4,50 m",7 t,foils,29 m,monotype,320 m2,570 m2,fabrice amedeo
1,FRA 49,"Gitana Eighty, Synerciel, Newrest-Matmut",Bruce Farr Design,Southern Ocean Marine (Nouvelle Zélande),08 Mars 2007,"18,28m","5,80m","4,50m",9t,2,28m,acier forgé,280 m2,560 m2,romain attanasio
2,FRA72,"Famille Mary-Etamine du Lys, Initiatives Coeur...",Marc Lombard,MAG France,01 Mars 1998,"18,28m","5,54m","4,50m",9t,2,29 m,acier,260 m2,580 m2,alexia barrier
3,17,Safran 2 - Des Voiles et Vous,Verdier - VPLP,CDK Technologies,12 Mars 2015,"18,28 m","5,80 m","4,50 m",8 t,foils,29 m,acier mécano soudé,310 m2,550 m2,yannick bestaven
4,08,NaN,VPLP,CDK Technologies,18 Août 2018,"18,28 m","5,85 m","4,50 m",8t,foils,29 m,acier,320 m2,600 m2,jeremie beyou
